In [3]:
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import OutputType
import pandas as pd
import datetime as datetime
from datetime import datetime, timedelta
from dateutil.parser import parse
from pulp import *
import numpy as np

In [4]:
for m in range(1, 3):
    if m == 2:
        for d in range(1, 29):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)
    else:
        for d in range(1, 32):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)

In [5]:
client.player_box_scores(day=1, month=1, year=2020)

[{'slug': 'jamesle01',
  'name': 'LeBron James',
  'team': <Team.LOS_ANGELES_LAKERS: 'LOS ANGELES LAKERS'>,
  'location': <Location.HOME: 'HOME'>,
  'opponent': <Team.PHOENIX_SUNS: 'PHOENIX SUNS'>,
  'outcome': <Outcome.WIN: 'WIN'>,
  'seconds_played': 2284,
  'made_field_goals': 11,
  'attempted_field_goals': 21,
  'made_three_point_field_goals': 1,
  'attempted_three_point_field_goals': 4,
  'made_free_throws': 8,
  'attempted_free_throws': 12,
  'offensive_rebounds': 2,
  'defensive_rebounds': 11,
  'assists': 12,
  'steals': 2,
  'blocks': 1,
  'turnovers': 5,
  'personal_fouls': 0,
  'game_score': 29.9},
 {'slug': 'antetgi01',
  'name': 'Giannis Antetokounmpo',
  'team': <Team.MILWAUKEE_BUCKS: 'MILWAUKEE BUCKS'>,
  'location': <Location.HOME: 'HOME'>,
  'opponent': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>,
  'outcome': <Outcome.WIN: 'WIN'>,
  'seconds_played': 1889,
  'made_field_goals': 13,
  'attempted_field_goals': 22,
  'made_three_point_field_goals': 3,
  'atte

In [6]:
all_tables = []
for m in range(1, 3):
    if m == 2:
        for d in range(1, 29):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            table  = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

    else:
        for d in range(1, 32):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            table  = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

In [7]:
full_df = all_tables[0]
for i in range(1, len(all_tables)):
    current_table = all_tables[i]
    full_df = full_df.append(current_table)
full_df.to_csv("./OutputCSVs/all_games.csv")

In [8]:
df = pd.read_csv("./OutputCSVs/teams_stats.csv")
df["Team"] = df["Team"].str.upper()
all_games = pd.read_csv("./OutputCSVs/all_games.csv")
x = all_games[["team", "opponent"]].loc[0]["team"]
df[df["Team"] == x]["TOV%"]

team_def = []
team_pace = []
team_tov = []
opp_def = []
opp_pace = []
opp_tov = []
all_games_teams = all_games[["team", "opponent"]]

for i in range(len(all_games_teams)):
    game = all_games_teams.loc[i]
    team = game["team"]
    opponent = game["opponent"]
    team_def.append(df[df["Team"] == team]["DRtg"].iloc[0])
    team_pace.append(df[df["Team"] == team]["Pace"].iloc[0])
    team_tov.append(df[df["Team"] == team]["TOV%"].iloc[0])
    opp_def.append(df[df["Team"] == opponent]["DRtg"].iloc[0])
    opp_pace.append(df[df["Team"] == opponent]["Pace"].iloc[0])
    opp_tov.append(df[df["Team"] == opponent]["TOV%"].iloc[0])
    
all_games["Team Defensive Rating"] = team_def
all_games["Team Pace"] = team_pace
all_games["Team Turnover %"] = team_tov
all_games["Opponent Defensive Rating"] = opp_def
all_games["Opponent Pace"] = opp_pace
all_games["Opponent Turnover %"] = opp_tov

all_games.to_csv("./OutputCSVs/all_games_new.csv")
df = pd.read_csv("./OutputCSVs/all_games_new.csv")
attempted_2s = df["attempted_field_goals"] - df["attempted_three_point_field_goals"]
made_2s = df["made_field_goals"] - df["made_three_point_field_goals"]
rebounds = df["offensive_rebounds"] + df["defensive_rebounds"]
at_home = df["location"] == "HOME"
df["attempted_two_point_field_goals"] = attempted_2s
df["made_two_point_field_goals"] = made_2s
df["total_rebounds"] = rebounds
df["at_home"] = at_home
df.to_csv("./OutputCSVs/all_games_updated.csv")
all_games_actual = pd.read_csv("./OutputCSVs/all_games_updated.csv")
all_games_actual =  all_games_actual.iloc[:, 3:].reset_index()

In [9]:
def get_stats(player, date, number_rows):
    converted_datetime = datetime.strptime(date, '%Y-%m-%d')
    player_rows = all_games_actual.loc[all_games_actual['name'] == player]
    #print(player_rows)
    selected_rows = []
    if (len(player_rows)) < number_rows:
        for i in range(len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
        return pd.DataFrame(selected_rows)
    index = 0
    for i in range(len(player_rows)):
        curr_date = player_rows.iloc[i]['Date']
        if datetime.strptime(curr_date, '%Y-%m-%d') >= converted_datetime:
            index = i - 1
            break
    if index != 0:
        if index + 1 - number_rows < 0:
            for i in range(index + 1):
                selected_rows.append(player_rows.iloc[i])
        else:
            for i in range(index + 1 - number_rows, index + 1):
                selected_rows.append(player_rows.iloc[i])
#     elif converted_datetime >= players_rows[len(player_rows) - 1]:
    else:
        for i in range(len(player_rows) - number_rows, len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
    return pd.DataFrame(selected_rows)

In [10]:
def get_points(row_data):
    three_pt_fgs = row_data['made_three_point_field_goals']
    two_pt_fgs = row_data['made_two_point_field_goals']
    made_fts = row_data['made_free_throws']
    total_rebounds = row_data['rebounds']
    assists = row_data['assists']
    blocks = row_data['blocks']
    steals = row_data['steals']
    turnovers = row_data['turnovers']
    FD_points = three_pt_fgs * 3 + two_pt_fgs * 2 + made_fts + total_rebounds * 1.2 + assists * 1.5 + blocks * 3 + steals * 3 - turnovers
    FD_dollars = FD_points * 200
    return (FD_points, FD_dollars)

In [11]:
def weight_function(statistic, weight):
    s = 0
    if type(statistic) == np.ndarray:
        for i in range(len(statistic)):
            s += statistic[len(statistic) - i - 1] * (weight ** i)
    else:
        for i in range(len(statistic)):
            s += statistic.iloc[len(statistic) - i - 1,] * (weight ** i)
    return s

In [12]:
def time_weighted_average(rows, statistic, weight):
    if rows.empty:
        return 0
    stat = rows[statistic]
    this_num = 1 / weight_function(np.ones(len(stat)), weight)
    return this_num * weight_function(stat, weight)
player_box_scores = pd.read_csv('./OutputCSVs/all_games_updated.csv')

input_statistics = ["name", "team", "date", "location", "opponent", "made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals"]

In [64]:
def statline_predictor(player_box_scores, input_statistics, sample_size = 5, weight = .8):
    player_box_scores = player_box_scores[~player_box_scores.index.duplicated()]
    player_box_scores.reindex(range(len(player_box_scores)), axis = "index")
    predicted_statlines = pd.DataFrame(index = player_box_scores.index, columns = input_statistics).fillna(0).T
    for box_index in player_box_scores.index:
        box_score = player_box_scores.loc[box_index]
        player_name = box_score["name"]
        game_date = str(box_score["Date"])[:10]
        print(player_name)
        print(box_index)
        print(game_date)
        last_n_rows = get_stats(player_name, game_date, sample_size)
        weighted_stats = [player_name, box_score["team"], game_date, box_score["location"], box_score["opponent"]]
        for stat in input_statistics[5:]:
            weighted_stats.append(round(time_weighted_average(last_n_rows, stat, weight), 2))
        predicted_statlines[box_index] = weighted_stats
    return predicted_statlines.T

In [14]:
predictions_samplesize5 = statline_predictor(all_games_actual, input_statistics, sample_size = 5)
predictions_samplesize5.to_csv("./OutputCSVs/predicted_statlines_trailing_5_games.csv")
predictions_samplesize10 = statline_predictor(all_games_actual, input_statistics, sample_size = 10)
predictions_samplesize10.to_csv("./OutputCSVs/predicted_statlines_trailing_10_games.csv")

In [15]:
def rebound_predictor_5(weighted_stats):
    return 3.495 + .9489 * weighted_stats["offensive_rebounds_y"] + .2874 * weighted_stats["defensive_rebounds_y"]
def freethrow_predictor_5(weighted_stats):
    return 1.373 + .6217 * weighted_stats["attempted_free_throws_y"] + -.0373 * weighted_stats["attempted_field_goals_y"] -.3289 * (weighted_stats["location_y"] == "HOME")
def twopt_predictor_5(weighted_stats):
    return max(0, -5.395 + .6064 * weighted_stats["attempted_two_point_field_goals_y"] - .2542 * weighted_stats["attempted_field_goals_y"] + .071 * weighted_stats["Opponent Defensive Rating"])
def threept_predictor_5(weighted_stats):
    return .766 + .214 * weighted_stats["attempted_three_point_field_goals_y"]
def assist_predictor_5(weighted_stats):
    return 2.06 + .532 * weighted_stats["assists_y"] - .064 * weighted_stats["blocks_y"]
def block_predictor_5(weighted_stats):
    return .329 + .182 * weighted_stats["offensive_rebounds_y"] - .038 * weighted_stats["defensive_rebounds_y"] + .424 * weighted_stats["blocks_y"]
def steal_predictor_5(weighted_stats):
    return max(0, -.936 + .339 * weighted_stats["steals_y"] + .129 * weighted_stats["Opponent Turnover %"])
def turnover_predictor_5(weighted_stats):
    return max(0, -2.589 + .18 * weighted_stats["turnovers_y"] + .121 * weighted_stats["assists_y"] + .032 * weighted_stats["Opponent Defensive Rating"])

In [16]:
all_games_predicted_5 = pd.read_csv("./OutputCSVs/predicted_statlines_trailing_5_games.csv")
all_games_predicted_5 = all_games_predicted_5.rename(columns={"Unnamed: 0": "indexr"})
all_games_merged_5 = all_games_actual.merge(all_games_predicted_5, left_on="index", right_on="indexr")
all_games_predicted_10 = pd.read_csv("./OutputCSVs/predicted_statlines_trailing_10_games.csv")
all_games_predicted_10 = all_games_predicted_10.rename(columns={"Unnamed: 0": "indexr"})
all_games_merged_10 = all_games_actual.merge(all_games_predicted_10, left_on="index", right_on="indexr")
all_games_merged_5.to_csv("./OutputCSVs/true_and_predicted_statlines_5.csv")
all_games_merged_10.to_csv("./OutputCSVs/true_and_predicted_statlines_10.csv")

In [17]:
weighted_preds = pd.read_csv("./OutputCSVs/true_and_predicted_statlines_5.csv")

In [66]:
output_statistics = ["name", "team", "date", "location", "opponent", "made_two_point_field_goals", "made_three_point_field_goals", "made_free_throws", "rebounds", "assists", "blocks", "steals", "turnovers"]

def statline_output(weighted_predicted_lines):
    output_statlines = pd.DataFrame(index = weighted_predicted_lines.index, columns = output_statistics).fillna(0).T
    for box_index in weighted_predicted_lines.index:
        weighted_line = weighted_predicted_lines.loc[box_index]
        output_stats = [weighted_line["name_x"], weighted_line["team_x"], weighted_line["Date"], weighted_line["location_x"], weighted_line["opponent_x"]]
        output_stats.append(twopt_predictor_5(weighted_line))
        output_stats.append(threept_predictor_5(weighted_line))
        output_stats.append(freethrow_predictor_5(weighted_line))
        output_stats.append(rebound_predictor_5(weighted_line))
        output_stats.append(assist_predictor_5(weighted_line))
        output_stats.append(block_predictor_5(weighted_line))
        output_stats.append(steal_predictor_5(weighted_line))
        output_stats.append(block_predictor_5(weighted_line))
        output_statlines[box_index] = output_stats
    return output_statlines.T
output_picks = statline_output(weighted_preds)
full_test_output = output_picks[output_picks["date"] > "2020-01-01"]
full_test_output.to_csv("./OutputCSVs/predicted_statlines_jan_feb_2020.csv")

In [20]:
feb_march_data = pd.read_csv('./OutputCSVs/predicted_statlines_jan_feb_2020.csv')
FD_points_array = []
FD_dollars_array = []
sum_points = 0
counter = 0
for index, row in feb_march_data.iterrows():
    tuple_values = get_points(row)
    FD_points_array.append(tuple_values[0])
    FD_dollars_array.append(tuple_values[1])
    sum_points += tuple_values[0]
    counter += 1
feb_march_data['FanDuel_Points'] = FD_points_array
feb_march_data['FanDuel_Dollars'] = FD_dollars_array
feb_march_data.to_csv("./OutputCSVs/Jan_Feb_statline_with_FanDuel_dollars.csv")

In [21]:
def generate_optimal_lineup(players, positions, values, costs, budget):
    num_variables = len(players)
    
    lp = LpProblem("My LP Problem", pulp.LpMaximize)
    
    d = {}
    for i in range(0, num_variables):
        d[players[i]] = LpVariable(players[i], cat="Binary")
    
    objective = sum(np.array(values) * np.array(list(d.values())))
    lp += objective
    
    pg_constraint = 0
    sg_constraint = 0
    sf_constraint = 0
    pf_constraint = 0
    c_constraint = 0
    for i in range(0, len(positions)):
        if positions[i] == "PG":
            pg_constraint += d[players[i]]
        elif positions[i] == "SG":
            sg_constraint += d[players[i]]
        elif positions[i] == "SF":
            sf_constraint += d[players[i]]
        elif positions[i] == "PF":
            pf_constraint += d[players[i]]
        else:
            c_constraint += d[players[i]]
    lp += pg_constraint == 2
    lp += sg_constraint == 2
    lp += sf_constraint == 2
    lp += pf_constraint == 2
    lp += c_constraint == 1
    
    cost = sum(np.array(costs) * np.array(list(d.values())))
    lp += cost <= budget
    
    lp.solve()
    
    lineup = [variable.name for variable in lp.variables() if variable.varValue == 1]
    print(lineup)
    #for variable in lp.variables():
    #    print ("{} = {}".format(variable.name, variable.varValue))
    #print(pulp.value(lp.objective))

In [26]:
all_tables = []
def box_scores_for_range_of_days(start_date, end_date):
    start_month = start_date.month
    end_month = end_date.month
    for m in range(start_month, end_month + 1):
        if m == start_month:
            start_day = start_date.day
        else:
            start_day = 1
        if m == end_month:
            end_day = end_date.day
        else:
            if m == 2:
                end_day = 29
            elif m in [9, 4, 6, 11]:
                end_day = 31
            else:
                end_day = 32
        
        for d in range(start_day, end_day):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)
            table = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

    full_df = all_tables[0]
    for i in range(1, len(all_tables)):
        current_table = all_tables[i]
        full_df = full_df.append(current_table)
    return full_df

In [23]:
def make_predictions(start_date, end_date, output = True):
    input_statistics = ["name", "team", "date", "location", "opponent", "made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals"]
    full_df = box_scores_for_range_of_days(start_date, end_date)
    predicted_statlines = statline_output(statline_predictor(full_df, input_statistics))
    if output:
        output_filename = './AllCSVs/' + start_date.month + '_' + start_date.day + '_' + end_date.month + '_' + end_date.day + '_2020_box_scores.csv'
        predicted_statlines.to_csv(output_filename)
    return predicted_statlines

In [67]:
import pytz

def predict_next_day(start_date, end_date, game_date):
    season_schedule = client.season_schedule(season_end_year = game_date.year)
    schedule_on_date = [game for game in season_schedule if (game['start_time'] - timedelta(hours = 4)).day == game_date.day and game['start_time'].month == game_date.month]
    team_objs_on_date = [game["home_team"] for game in schedule_on_date] + [game["away_team"] for game in schedule_on_date]
    teams_on_date = [t.name for t in team_objs_on_date]
    predictions_for_date = make_predictions(start_date, end_date, output = False)
    players_on_date = [statline for statline in predictions_for_date if statline["team"] in teams_on_date]
    output_filename = './AllCSVs/predictions_for_' + game_date.month + '_' + game_date.day + '_' + start_date.month + '_' + start_date.day + '_' + end_date.month + '_' + end_date.day + '_2020.csv'
    players_on_date.to_csv(output_filename)
    return players_on_date
    print(head(predictions_for_date))
predict_next_day(datetime(2020, 1, 1), datetime(2020, 1, 11), datetime(2020, 1, 12))

LeBron James
0
2020-01-01
Giannis Antetokounmpo
1
2020-01-01
D.J. Augustin
2
2020-01-01
Kelly Oubre
3
2020-01-01
Mitchell Robinson
4
2020-01-01
Anthony Davis
5
2020-01-01
Hassan Whiteside
6
2020-01-01
Devin Booker
7
2020-01-01
Julius Randle
8
2020-01-01
Bradley Beal
9
2020-01-01
Carmelo Anthony
10
2020-01-01
Avery Bradley
11
2020-01-01
Shabazz Napier
12
2020-01-01
Markelle Fultz
13
2020-01-01
Frank Ntilikina
14
2020-01-01
Deandre Ayton
15
2020-01-01
Nikola Vučević
16
2020-01-01
Bobby Portis
17
2020-01-01
Kyle Kuzma
18
2020-01-01
Khem Birch
19
2020-01-01
Robert Covington
20
2020-01-01
Marcus Morris
21
2020-01-01
Terrence Ross
22
2020-01-01
CJ McCollum
23
2020-01-01
Donte DiVincenzo
24
2020-01-01
Jordan McRae
25
2020-01-01
Troy Brown
26
2020-01-01
Josh Okogie
27
2020-01-01
Ricky Rubio
28
2020-01-01
Wesley Iwundu
29
2020-01-01
Ish Smith
30
2020-01-01
Gorgui Dieng
31
2020-01-01
Brook Lopez
32
2020-01-01
Eric Bledsoe
33
2020-01-01
Evan Fournier
34
2020-01-01
Ersan İlyasova
35
2020-01-01
Gar

KeyError: 'name_x'

In [ ]:
def append_actual_results(start_date, end_date, game_date):
    predicted_results = predict_next_day(start_date, end_date, game_date)
    actual_results = box_scores_for_range_of_days(game_date, game_date)
    pred_outputs = [get_points(actual_results["name" == player_name]) for player_name in predicted_results["name"]]
    predicted_results["actual"] = pred_outputs
    output_filename = './AllCSVs/' + game_date.month + '_' + game_date.day + '_2020_box_scores_predicted_with_actual.csv'
    predicted_results.to_csv(output_filename)
    return predicted_results

In [ ]:
def alternate_contests(start_date, end_date, game_date, search_datetime, inclusive):
    season_schedule = client.season_schedule(season_end_year = game_date.year)
    schedule_on_date = [game for game in season_schedule if (game['start_time'] - timedelta(hours = 4)).day == game_date.day and game['start_time'].month == game_date.month]
    times_on_date = np.unique([game['start_time'] - timedelta(hours = 4) for game in schedule_on_date])
    if not inclusive:
        game_teams = [game["home_team"] for game in schedule_on_date if game['start_time'] == search_datetime] + [game["away_team"] for game in schedule_on_date if game['start_time'] == search_datetime]
    else:
        times_after_search = [t for t in times_on_date if 60 * t.hour + t.minute >= 60 * search_datetime.hour + search_datetime.minute]
        game_teams = [game["home_team"] for game in schedule_on_date if game['start_time'] in times_after_search] + [game["away_team"] for game in schedule_on_date if game['start_time'] in times_after_search]
    predictions_for_date = make_predictions(start_date, end_date, output = False)
    players_on_date = [statline for statline in predictions_for_date if statline["team"] in teams_on_date]
    output_filename = './AllCSVs/' + game_date.month + '_' + game_date.day + '_2020_alternate_' + search_datetime.hour + "_" + search_datetime.minute + '.csv'
    players_on_date.to_csv(output_filename)
    return players_on_date
alternate_contests(datetime(2020, 1, 1), datetime(2020, 1, 2), datetime(2020, 1, 3), datetime(2020, 1, 2, 21, 30), True)

In [ ]:
def optimal_lineup_games(input_games, ):